## Making activity spaces using open source Python libraries

Here we will try making activity space models directly from within Python enviornment. The code used here used open source Python libraries only. 

**Setting the paths**

As a first step (and a good coding practice) let's clearly define our paths so that we do not run into issues while working with data.

In [ ]:
import pathlib
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"
output_DIRECTORY = NOTEBOOK_PATH / "output"
raster_DIRECTORY = NOTEBOOK_PATH / "output/raster"

# paths to our data and outputs:
home_file = DATA_DIRECTORY / "home20.shp"
activity_file = DATA_DIRECTORY / "eep20.shp"
output_file = output_DIRECTORY /"Output.gpkg"
route_file = DATA_DIRECTORY /"shortest_routes.shp"
temp_DIRECTORY = NOTEBOOK_PATH / "output" / "temp"

**Loading data**

Now let's load our home data as a geodataframe and project to a suitable CRS. Then we take a quick look at data.

In [ ]:
gdf_home = gpd.read_file(home_file)
gdf_home=gdf_home.to_crs("EPSG:3067")

In [ ]:
gdf_home.head()

In [ ]:
gdf_home.plot()

Let's try buffer first:

In [ ]:
buffer_distance = 500
buffered_home = gpd.GeoDataFrame(geometry=gdf_home['geometry'].buffer(buffer_distance), crs=gdf_home.crs)
ax = buffered_home.plot(marker='o', color='white', markersize=100, figsize=(8, 8))
buffered_home.boundary.plot(ax=ax, edgecolor='blue', linewidth=2)
gdf_home.plot(ax=ax, facecolor='purple', markersize=10, marker='x')

Now let's use the activity space modelers. First we will try the home range model. There is a function in the main.py file called "nb_withRoute". This function creates a home range model by also taking the routes into account. let's call the function and provide the inputs:

In [ ]:
from main import nb_withRoute
D1, D2 = 500, 200
nb_withRoute(home_file, activity_file, route_file, output_file, "uid", D1, D2)

In [ ]:
# Lets load the output and take a look at it
home_range = gpd.read_file(output_DIRECTORY / "Output.gpkg")
home_range.head()


In [ ]:
home_range.boundary.plot()

In [ ]:
# make an interactive map and add basemap
import folium
interactive_map = folium.Map(
    location=(65, 25.48),
    zoom_start=10
)

hr = folium.features.GeoJson(
    home_range,
    name="Public transport stops"
)
hr.add_to(interactive_map)

interactive_map

In [ ]:
all_uids = home_range['uid'].tolist()
print (all_uids)

In [ ]:
# choose a uid to visualize alone
desired_uid = 920380
selected_row = home_range[home_range['uid'] == desired_uid]
selected_row.boundary.plot()

In [ ]:
from main import IREM
home_file = DATA_DIRECTORY / "small_data/Home.shp"
activity_file = DATA_DIRECTORY / "small_data/eep.shp"
route_file = DATA_DIRECTORY /"small_data/routes.shp"
output_file = output_DIRECTORY /"HR_small_data.gpkg"
D1, D2 = 500 , 200
nb_withRoute(home_file, activity_file, route_file, output_file, "uid", D1, D2)

In [ ]:
#IREM(home_file,activity_file,output_file,route_file)
pixelsize= 20
IREM(home_file,activity_file,output_file,route_file, pixelsize)

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Lets choose a uid to visualize 

uid = 425004
with rasterio.open(raster_DIRECTORY / f'nb_{uid}.tif') as src:
    # Read the raster data
    raster_data = src.read(1)  # Use index 1 for a single-band raster

    # Define the margin
    margin = 100  # Adjust this value based on your preference

    # Calculate the new extent with a margin
    left = src.bounds.left - margin * src.res[0]
    right = src.bounds.right + margin * src.res[0]
    bottom = src.bounds.bottom - margin * src.res[1]
    top = src.bounds.top + margin * src.res[1]

    # Visualize the raster data with the new extent and add a title
    plt.figure(figsize=(10, 10))
    plt.imshow(raster_data, extent=(left, right, bottom, top), cmap='viridis')  # You can choose a different colormap (cmap)
    plt.title(f'IREM for uid:{uid}')
    plt.colorbar(label='Exposure Value')  # Add a colorbar if needed
    plt.show()
